In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [12]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

In [2]:
import pandas as pd
import re

/Users/lingyizhao/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
from datasets import load_dataset

dataset = load_dataset("SALT-NLP/silent_signals_detection", split="train")
df_detection = dataset.to_pandas()
df_detection

/Users/lingyizhao/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,idx,dog_whistle,dog_whistle_root,ingroup,definition,example,label
0,0,YWNBAW,YWNBAW,transphobic,You Will Never Be A Woman (YWNBAW) is an alt-r...,"""Might I suggest you reach out to YWNBAW? Very...",coded
1,1,109,109,antisemitic,109 stands for the 109 locations that Jews wer...,"""""Wow 109 countries, and they consider us as ...",coded
2,2,alarmist,alarmist,climate change denier,"""Alarmist"" is a common global warming denialis...","""A mountain of evidence shows climate alarmist...",coded
3,3,Aiden,Aiden,transphobic,"Derogatory term for a transgender man, based o...","""TRAs always need to get the last word in and ...",coded
4,4,Judeo-Christian,Judeo-Christian,Islamophobic,"It might seem neighbourly, even pluralistic, t...","""We are stopping cold the attacks on Judeo-Chr...",coded
...,...,...,...,...,...,...,...
96,96,norm20,None,None,None,"As the plane accelerates, we feel like we’re b...",non-coded
97,97,norm21,None,None,None,"Depending on the type of bomb, weapon safety r...",non-coded
98,98,norm22,None,None,None,"and to ad on, even humans don't die of ""old ag...",non-coded
99,99,norm23,None,None,None,I just don't like it anymore. I lived in Wisco...,non-coded


In [5]:
from together import Together
import os
os.environ["TOGETHER_API_KEY"] = "f32fc2d24a5ec68f0b440ad698afe50cf2d5baf6f8a3384d5e45c55abfff751e"
client = Together()

In [ ]:
# zero shot
results = []

for sentence in df_detection["example"][0:51]:
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[
            {
                "role": "user",
                "content": (
                    "You are a helpful assistant participating in a content moderation task of dog whistle detection. "
                    "A dog whistle is the use of coded or suggestive language in political messaging to garner support from a particular group without provoking opposition. "
                    "Does the following sentence contain a dog whistle? If it does, please identify the dog whistle.\n\n"
                    f"{sentence}\n\n"
                    "Please structure your response as a JSON object with the following format:\n"
                    "Make sure all values are properly quoted and escaped, and that the entire output is valid JSON.\n\n"
                    "{\n"
                    "  \"sentence\": \"<original sentence here>\",\n"
                    "  \"dog_whistle_present\": true or false,\n"
                    "  \"dog_whistle_term\": \"<term if present, else null>\",\n"
                    "  \"coded_meaning\": \"<coded meaning if present, else null>\"\n"
                    "}"
                )
            }
        ]
    )

    raw_response = response.choices[0].message.content
    print(raw_response)
    results.append({
        "raw_response": raw_response
    })


In [26]:
### few shot

def llm_detection_few(sentence, example_sentences, example_answers, example_terms, example_meanings):
    input_definition = f"""You are a helpful assistant participating in a content moderation task of dog whistle detection.
    Here are some examples:"""
    for i in range(len(example_sentences)):
        input_definition += f"""Sentence: {example_sentences[i]}
                               Answer: {example_answers[i]}
                               Dog Whistle Term (if applicable): {example_terms[i]}
                               Meaning of Dog Whistle (if applicable) : {example_meanings[i]}
        """
    input_definition += f"""A dog whistle is the use of coded or suggestive language in political messaging to garner support from a particular group without provoking opposition. 
    Does the following sentence contain a dog whistle?
    If it does, please identify the dog whistle.

    {sentence}
    
    "Please structure your response as a JSON object with the following format:\n"
                    "Make sure all values are properly quoted and escaped, and that the entire output is valid JSON.\n\n"
                    "{{\n"
                    "  \"sentence\": \"<original sentence here>\",\n"
                    "  \"dog_whistle_present\": true or false,\n"
                    "  \"dog_whistle_term\": \"<term if present, else null>\",\n"
                    "  \"coded_meaning\": \"<coded meaning if present, else null>\"\n"
                    "}}"
    
    """
    print(input_definition, "\n")
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": input_definition}]
    )
    raw_output = response.choices[0].message.content
    raw_fewshot_result.append({
        "raw_fewshot_response": raw_output
    })
    print(raw_fewshot_result, "\n")
    #match = re.search(r"```json\s*(.*?)```", raw_output, flags=re.DOTALL)
    #cleaned = match.group(1) if match else ""
    return raw_fewshot_result

In [17]:
from tqdm import tqdm
import numpy as np
import json

In [ ]:
raw_fewshot_result=[]

for index, row in tqdm(df_detection.iterrows()):
    dog_whistle_not_to_use = row['dog_whistle']
    df_example = df_detection[df_detection['dog_whistle'] != dog_whistle_not_to_use]
    df_example = df_example.iloc[np.random.randint(0, len(df_example), size=3),:]
    raw_fewshot_result = llm_detection_few(row['example'],
      list(df_example['example']),
      list(df_example['label']),
      list(df_example['dog_whistle']),
      list(df_example['definition']))


In [30]:
with open("llama3.0_dog_whistle_fewshot_responses.json", "w") as f:
    json.dump(raw_fewshot_result, f, indent=2)

In [36]:
def parce_detection_result(raw_data):

    # Prepare a list to store parsed outputs
    parsed_results = []

    # Loop through each response
    for entry in raw_data:
        raw_response = entry["raw_fewshot_response"].strip()# fewshot

        # Remove triple backticks if present
        if raw_response.startswith("```") and raw_response.endswith("```"):
            raw_response = "\n".join(raw_response.splitlines()[1:-1]).strip()

        try:
            # First pass: convert the escaped string into a raw JSON string
            if raw_response.startswith("{") and isinstance(raw_response, str):
                first_pass = json.loads(raw_response)
            else:
                first_pass = raw_response

            # Second pass: convert the actual JSON object
            if isinstance(first_pass, str):
                parsed_json = json.loads(first_pass)
            else:
                parsed_json = first_pass

            parsed_results.append(parsed_json)

        except json.JSONDecodeError:
            parsed_results.append({
                "sentence": None,
                "dog_whistle_present": None,
                "dog_whistle_term": None,
                "coded_meaning": None
            })


    # Convert list of dicts to DataFrame
    df_responses = pd.DataFrame(parsed_results)
    return df_responses

# Optional: save to CSV


#df_fewshot_responses

In [34]:
# Load the JSON file
with open("llama3.0_dog_whistle_fewshot_responses.json", "r") as file:
    raw_data = json.load(file)

In [37]:
df_fewshot_responses=parce_detection_result(raw_data)
df_fewshot_responses

,sentence,dog_whistle_present,dog_whistle_term,coded_meaning
0,Might I suggest you reach out to YWNBAW? Very ...,True,YWNBAW,"YWNBAW is an acronym that stands for ""You Will..."
1,"Wow 109 countries, and they consider us as ca...",True,blood libel,Anti-Semitic conspiracy theory
2,A mountain of evidence shows climate alarmists...,True,climate alarmists,A term used to dismiss and undermine the credi...
3,TRAs always need to get the last word in and n...,True,TRAs,"Trans Rights Activists, implying that individu..."
4,We are stopping cold the attacks on Judeo-Chri...,True,Judeo-Christian values,Exclusion of non-Christian or non-Jewish peopl...
...,...,...,...,...
96,"As the plane accelerates, we feel like we’re b...",False,None,None
97,"Depending on the type of bomb, weapon safety r...",False,None,None
98,"and to ad on, even humans don't die of ""old ag...",False,None,None
99,I just don't like it anymore. I lived in Wisco...,False,None,None


In [38]:
df_fewshot_responses.to_csv("parsed_dog_whistle_fewshot_responses.csv", index=False)

In [3]:
df_fewshot_responses=pd.read_csv("parsed_dog_whistle_fewshot_responses.csv")

In [6]:
df_fewshot_all = pd.concat([df_detection, df_fewshot_responses.reset_index(drop=True)], axis=1)
df_fewshot_all.drop(columns=["sentence"], inplace=True)

In [7]:
df_fewshot_all

,idx,dog_whistle,dog_whistle_root,ingroup,definition,example,label,dog_whistle_present,dog_whistle_term,coded_meaning
0,0,YWNBAW,YWNBAW,transphobic,You Will Never Be A Woman (YWNBAW) is an alt-r...,"""Might I suggest you reach out to YWNBAW? Very...",coded,True,YWNBAW,"YWNBAW is an acronym that stands for ""You Will..."
1,1,109,109,antisemitic,109 stands for the 109 locations that Jews wer...,"""""Wow 109 countries, and they consider us as ...",coded,True,blood libel,Anti-Semitic conspiracy theory
2,2,alarmist,alarmist,climate change denier,"""Alarmist"" is a common global warming denialis...","""A mountain of evidence shows climate alarmist...",coded,True,climate alarmists,A term used to dismiss and undermine the credi...
3,3,Aiden,Aiden,transphobic,"Derogatory term for a transgender man, based o...","""TRAs always need to get the last word in and ...",coded,True,TRAs,"Trans Rights Activists, implying that individu..."
4,4,Judeo-Christian,Judeo-Christian,Islamophobic,"It might seem neighbourly, even pluralistic, t...","""We are stopping cold the attacks on Judeo-Chr...",coded,True,Judeo-Christian values,Exclusion of non-Christian or non-Jewish peopl...
...,...,...,...,...,...,...,...,...,...,...
96,96,norm20,None,None,None,"As the plane accelerates, we feel like we’re b...",non-coded,False,NaN,NaN
97,97,norm21,None,None,None,"Depending on the type of bomb, weapon safety r...",non-coded,False,NaN,NaN
98,98,norm22,None,None,None,"and to ad on, even humans don't die of ""old ag...",non-coded,False,NaN,NaN
99,99,norm23,None,None,None,I just don't like it anymore. I lived in Wisco...,non-coded,False,NaN,NaN


In [8]:
df_fewshot_all['dog_whistle_present'] = df_fewshot_all['dog_whistle_present'].map({True:'coded', False:'non-coded'})

In [62]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [46]:
accuracy_fewshot=accuracy_score(df_fewshot_all['label'], df_fewshot_all['dog_whistle_present'])
f1_fewshot=f1_score(df_fewshot_all['label'], df_fewshot_all['dog_whistle_present'], pos_label='coded')
recall_fewshot=recall_score(df_fewshot_all['label'], df_fewshot_all['dog_whistle_present'], pos_label='coded')
precision_score=precision_score(df_fewshot_all['label'], df_fewshot_all['dog_whistle_present'], pos_label='coded')
print("fewshot_accuracy: ",accuracy_fewshot)
print("f1_fewshot: ", f1_fewshot)
print("recall_fewshot: ",recall_fewshot)
print("precision_score: ",precision_score)

fewshot_accuracy:  0.8613861386138614
f1_fewshot:  0.8750000000000001
recall_fewshot:  0.9607843137254902
precision_score:  0.8032786885245902


In [15]:
def clean_text(text):
    if isinstance(text, str):
        # Lowercase
        text = text.lower()
        # Remove special characters (keep alphanumeric and spaces)
        text = re.sub(r'[^a-z0-9\s]', '', text)
        return text.strip()
    return text

df_fewshot_all['dog_whistle_term'] = df_fewshot_all['dog_whistle_term'].apply(clean_text)
df_fewshot_all['dog_whistle'] = df_fewshot_all['dog_whistle'].apply(clean_text)

detection_outputs_coded = df_fewshot_all[
    (df_fewshot_all['label'] == 'coded') & (df_fewshot_all['dog_whistle_present'] == 'coded')
]
detection_outputs_coded

detection_outputs_coded['same_meaning'] = detection_outputs_coded.apply(
    lambda row: (row['dog_whistle_term'] in row['dog_whistle']) or 
                (row['dog_whistle'] in row['dog_whistle_term']), axis=1)
sum(detection_outputs_coded['same_meaning']) / len(detection_outputs_coded)

/var/folders/m9/_gpbs0bn61q7077ldd342jc00000gn/T/ipykernel_34259/2770942077.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detection_outputs_coded['same_meaning'] = detection_outputs_coded.apply(


0.5306122448979592

In [9]:
# Step 1: Create a boolean mask for the relevant subset
mask = (df_fewshot_all['label'] == 'coded') & (df_fewshot_all['dog_whistle_present'] == 'coded')

# Step 2: Compute the 'same_meaning' column only for the matching rows
df_fewshot_all.loc[mask, 'same_meaning'] = df_fewshot_all.loc[mask].apply(
    lambda row: (row['dog_whistle_term'] in row['dog_whistle']) or 
                (row['dog_whistle'] in row['dog_whistle_term']),
    axis=1
)


In [13]:
df_fewshot_all

,idx,dog_whistle,dog_whistle_root,ingroup,definition,example,label,dog_whistle_present,dog_whistle_term,coded_meaning,same_meaning
0,0,YWNBAW,YWNBAW,transphobic,You Will Never Be A Woman (YWNBAW) is an alt-r...,"""Might I suggest you reach out to YWNBAW? Very...",coded,coded,YWNBAW,"YWNBAW is an acronym that stands for ""You Will...",True
1,1,109,109,antisemitic,109 stands for the 109 locations that Jews wer...,"""""Wow 109 countries, and they consider us as ...",coded,coded,blood libel,Anti-Semitic conspiracy theory,False
2,2,alarmist,alarmist,climate change denier,"""Alarmist"" is a common global warming denialis...","""A mountain of evidence shows climate alarmist...",coded,coded,climate alarmists,A term used to dismiss and undermine the credi...,True
3,3,Aiden,Aiden,transphobic,"Derogatory term for a transgender man, based o...","""TRAs always need to get the last word in and ...",coded,coded,TRAs,"Trans Rights Activists, implying that individu...",False
4,4,Judeo-Christian,Judeo-Christian,Islamophobic,"It might seem neighbourly, even pluralistic, t...","""We are stopping cold the attacks on Judeo-Chr...",coded,coded,Judeo-Christian values,Exclusion of non-Christian or non-Jewish peopl...,True
...,...,...,...,...,...,...,...,...,...,...,...
96,96,norm20,None,None,None,"As the plane accelerates, we feel like we’re b...",non-coded,non-coded,NaN,NaN,NaN
97,97,norm21,None,None,None,"Depending on the type of bomb, weapon safety r...",non-coded,non-coded,NaN,NaN,NaN
98,98,norm22,None,None,None,"and to ad on, even humans don't die of ""old ag...",non-coded,non-coded,NaN,NaN,NaN
99,99,norm23,None,None,None,I just don't like it anymore. I lived in Wisco...,non-coded,non-coded,NaN,NaN,NaN


In [22]:
df_fewshot_all.to_csv("detection_fewshot_v2.csv", index=False)

In [17]:
import numpy as np

In [18]:
model = SentenceTransformer('all-MiniLM-L6-v2')
coded_embeddings = model.encode(detection_outputs_coded['coded_meaning'].tolist(), convert_to_numpy=True)
definition_embeddings = model.encode(detection_outputs_coded['definition'].tolist(), convert_to_numpy=True)
cosine_sims = []
l2_dists = []
l1_dists = []

# Loop over pairs
for c, d in zip(coded_embeddings, definition_embeddings):
    cosine_sims.append(1 - cosine(c, d))
    l2_dists.append(np.linalg.norm(c - d))          # Euclidean
    l1_dists.append(np.sum(np.abs(c - d)))          # Manhattan

average_cosine_sim = np.mean(cosine_sims)
average_l2_dist = np.mean(l2_dists)
average_l1_dist = np.mean(l1_dists)

print("Average cosine similarity:", average_cosine_sim)
print("Average L2 distance (Euclidean):", average_l2_dist)
print("Average L1 distance (Manhattan):", average_l1_dist)

Average cosine similarity: 0.4885385475900708
Average L2 distance (Euclidean): 0.99662405
Average L1 distance (Manhattan): 15.495332


In [19]:
detection_outputs_coded['cosine_similarity'] = cosine_sims
detection_outputs_coded['l2_distance'] = l2_dists
detection_outputs_coded['l1_distance'] = l1_dists

/var/folders/m9/_gpbs0bn61q7077ldd342jc00000gn/T/ipykernel_34259/758581730.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detection_outputs_coded['cosine_similarity'] = cosine_sims
/var/folders/m9/_gpbs0bn61q7077ldd342jc00000gn/T/ipykernel_34259/758581730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detection_outputs_coded['l2_distance'] = l2_dists
/var/folders/m9/_gpbs0bn61q7077ldd342jc00000gn/T/ipykernel_34259/758581730.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [20]:
# Step 1: Create a mapping from example → cosine_similarity
cosine_sim_map = detection_outputs_coded.set_index('example')['cosine_similarity'].to_dict()

# Step 2: Map the values back to df_combined using the same key
df_fewshot_all['cosine_similarity'] = df_fewshot_all['example'].map(cosine_sim_map)


In [21]:
df_fewshot_all

,idx,dog_whistle,dog_whistle_root,ingroup,definition,example,label,dog_whistle_present,dog_whistle_term,coded_meaning,same_meaning,cosine_similarity
0,0,ywnbaw,YWNBAW,transphobic,You Will Never Be A Woman (YWNBAW) is an alt-r...,"""Might I suggest you reach out to YWNBAW? Very...",coded,coded,ywnbaw,"YWNBAW is an acronym that stands for ""You Will...",True,0.782862
1,1,109,109,antisemitic,109 stands for the 109 locations that Jews wer...,"""""Wow 109 countries, and they consider us as ...",coded,coded,blood libel,Anti-Semitic conspiracy theory,False,0.372069
2,2,alarmist,alarmist,climate change denier,"""Alarmist"" is a common global warming denialis...","""A mountain of evidence shows climate alarmist...",coded,coded,climate alarmists,A term used to dismiss and undermine the credi...,True,0.636309
3,3,aiden,Aiden,transphobic,"Derogatory term for a transgender man, based o...","""TRAs always need to get the last word in and ...",coded,coded,tras,"Trans Rights Activists, implying that individu...",False,0.454018
4,4,judeochristian,Judeo-Christian,Islamophobic,"It might seem neighbourly, even pluralistic, t...","""We are stopping cold the attacks on Judeo-Chr...",coded,coded,judeochristian values,Exclusion of non-Christian or non-Jewish peopl...,True,0.671445
...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,norm20,None,None,None,"As the plane accelerates, we feel like we’re b...",non-coded,non-coded,NaN,NaN,NaN,NaN
97,97,norm21,None,None,None,"Depending on the type of bomb, weapon safety r...",non-coded,non-coded,NaN,NaN,NaN,NaN
98,98,norm22,None,None,None,"and to ad on, even humans don't die of ""old ag...",non-coded,non-coded,NaN,NaN,NaN,NaN
99,99,norm23,None,None,None,I just don't like it anymore. I lived in Wisco...,non-coded,non-coded,NaN,NaN,NaN,NaN


In [114]:
dataset = load_dataset("SALT-NLP/silent_signals_disambiguation", split="train")
df_disambiguation = dataset.to_pandas()

In [115]:
df_disambiguation

,type,dog_whistle,dog_whistle_root,ingroup,content,date,speaker,chamber,reference,subreddit,label,definition
0,colloquial,terrorist,Islamic terrorists,Islamophobic,Radical feminism is a terrorist organization,None,None,None,None,walkaway,non-coded,Muslim people as a whole are a threat
1,colloquial,terrorist,Islamic terrorists,Islamophobic,There's a group of right-wing terrorists that ...,None,None,None,None,walkaway,non-coded,Muslim people as a whole are a threat
2,colloquial,terrorist,Islamic terrorists,Islamophobic,Would be difficult to do. About the only mecha...,None,None,None,None,consoom,non-coded,Muslim people as a whole are a threat
3,colloquial,terrorist,Islamic terrorists,Islamophobic,"Rapes, terrorist attacks, violence in general ...",None,None,None,None,walkaway,non-coded,Muslim people as a whole are a threat
4,colloquial,based,based,white supremacist,Where are the prosecutions for a doctor taking...,None,None,None,None,walkaway,non-coded,Endorsement of someone promoting racism or oth...
...,...,...,...,...,...,...,...,...,...,...,...,...
119,formal,election integrity,election integrity,racist,S. Senate spoke of ``the pernicious and nasty...,2020-01-06,Mr. BARRASSO,Senate,CREC-2021-03-03-pt1-PgS1007,None,noncoded,Voter suppression that targets people of color
120,formal,election integrity,election integrity,racist,"Earlier this year, we witnessed only the lates...",2020-01-06,Mr. SCHUMER,Senate,CREC-2021-03-17-pt1-PgS1578-2,None,noncoded,Voter suppression that targets people of color
121,formal,election integrity,election integrity,racist,One is the breakdown of trust in the system it...,2020-01-06,Mr. KING,Senate,CREC-2021-10-19-pt1-PgS7082-5,None,noncoded,Voter suppression that targets people of color
122,formal,election integrity,election integrity,racist,It was so disturbing to read in the Washington...,2020-01-06,Mr. SCHUMER,Senate,CREC-2023-02-01-pt1-PgS180-2,None,noncoded,Voter suppression that targets people of color


### disambiguation

In [49]:
raw_disambiguation_result=[]

In [68]:
def llm_disambiguation(lst_sentences, sentence_indexes, dog_whistle, definition_of_dog_whistle):
    input_definition = f"""You are a helpful assistant participating in a content moderation task of dog whistle detection.
        A dogwhistle is the use of coded or suggestive language in political messaging to garner support 
        from a particular group without provoking opposition. 
        The coded meaning of {dog_whistle} is: {definition_of_dog_whistle}.  
        Can you identify which sentences in the set below are using {dog_whistle} as a dog whistle? 
        {[str(sentence_indexes[key])+': ' + str(value) for key, value in enumerate(lst_sentences)]}
        Please structure your response as a JSON object,
        where {{
        "sentences_with_dog_whistles": "<[list of indexes]>",
        "explanations": "<explanation>"
        }}.
        """
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": input_definition}]
    )
    raw_output = response.choices[0].message.content
    raw_disambiguation_result.append({
        "raw_disambiguation_result": raw_output
    })

    match = re.search(r"```json\s*(.*?)```", raw_output, flags=re.DOTALL)
    cleaned = match.group(1) if match else ""
    
    return cleaned, raw_disambiguation_result

In [69]:
results_disambiguation=[]
for dog_whistle in np.unique(df_disambiguation['dog_whistle']):
    definition_of_dog_whistle = list(df_disambiguation.loc[df_disambiguation['dog_whistle']==dog_whistle, 'definition'])[0]
    lst_sentences = list(df_disambiguation.loc[df_disambiguation['dog_whistle']==dog_whistle, 'content'])
    sentence_indexes = list(df_disambiguation[df_disambiguation['dog_whistle'] == dog_whistle].index)
    cleaned_result, raw_disambiguation_result = llm_disambiguation(lst_sentences, sentence_indexes, dog_whistle, definition_of_dog_whistle)
    try:
      response_dict = json.loads(cleaned_result)
      results_disambiguation.append(response_dict)
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON at row {index}: {e}") 
      results_disambiguation.append({
      "sentences_with_dog_whistles": np.nan,
      "explanations": np.nan
    })
with open('llama3.0_disambiguation.json', 'w') as outfile:
  json.dump(results_disambiguation, outfile, indent=4)

with open('llama3.0_disambiguation_raw.json', 'w') as outfile:
  json.dump(raw_disambiguation_result, outfile, indent=4)

Error decoding JSON at row 100: Expecting value: line 1 column 1 (char 0)


In [54]:
np.unique(df_disambiguation['dog_whistle'])

array(['Baltimore', 'Islamic extremism', 'Islamic terrorism', 'based',
       'echo', 'election integrity', 'extremism', 'extremist',
       'extremists', 'fatherless', 'fatherless home', 'globalist',
       'globalists', 'grooming', 'hygienic', 'jogger', 'law and order',
       'soy', 'steroids', 'terrorist', 'welfare'], dtype=object)

In [70]:
with open('llama3.0_disambiguation.json', 'r') as f:
  data = json.load(f)
disambiguation_results = pd.DataFrame(data)
disambiguation_results

,sentences_with_dog_whistles,explanations
0,[105],Sentence 105 uses 'Baltimore' as a dog whistle...
1,[82],The sentence is using Islamic extremism as a d...
2,[0],The sentence is using 'radical Islamic terrori...
3,"[37, 38, 39, 40, 41]",These sentences are using 'based' as a dog whi...
4,NaN,NaN
5,"[115, 117, 118, 120, 121, 122, 123]",These sentences use 'election integrity' as a ...
6,[77],Sentence 77 is using extremism as a dog whistl...
7,[0],The sentence 'As head of the Department of Hea...
8,"[80, 81]",Both sentences use the term 'extremists' in a ...
9,"[27, 62, 63, 65]",These sentences use 'fatherless' as a dog whis...


In [75]:
disambiguation_results.loc[4, "sentences_with_dog_whistles"]=[]
disambiguation_results.loc[4, "explanations"]="The word 'echo' in the sentence is used in its standard sense to indicate that sentiments are similar, not as a dog whistle to target any group."

In [76]:
disambiguation_results.to_csv("disambiguation_results.csv", index=False)

In [77]:
import ast
df_disambiguation['predicted_label'] = np.repeat('noncoded', repeats=len(df_disambiguation))
for row_num, row in disambiguation_results.iterrows():
    lst_positive_preds = row['sentences_with_dog_whistles']
    if isinstance(lst_positive_preds, str):
        lst_positive_preds = ast.literal_eval(lst_positive_preds)
    lst_positive_preds = [int(index) for index in lst_positive_preds]
    df_disambiguation.loc[df_disambiguation.index.isin(lst_positive_preds), 'predicted_label'] = 'coded'
df_disambiguation['label'] = df_disambiguation['label'].replace({
    'non-coded': 'noncoded',
    'noncoded': 'noncoded'  # Just in case of inconsistent hyphenation
})
df_disambiguation

,type,dog_whistle,dog_whistle_root,ingroup,content,date,speaker,chamber,reference,subreddit,label,definition,predicted_label
0,colloquial,terrorist,Islamic terrorists,Islamophobic,Radical feminism is a terrorist organization,None,None,None,None,walkaway,noncoded,Muslim people as a whole are a threat,coded
1,colloquial,terrorist,Islamic terrorists,Islamophobic,There's a group of right-wing terrorists that ...,None,None,None,None,walkaway,noncoded,Muslim people as a whole are a threat,noncoded
2,colloquial,terrorist,Islamic terrorists,Islamophobic,Would be difficult to do. About the only mecha...,None,None,None,None,consoom,noncoded,Muslim people as a whole are a threat,noncoded
3,colloquial,terrorist,Islamic terrorists,Islamophobic,"Rapes, terrorist attacks, violence in general ...",None,None,None,None,walkaway,noncoded,Muslim people as a whole are a threat,coded
4,colloquial,based,based,white supremacist,Where are the prosecutions for a doctor taking...,None,None,None,None,walkaway,noncoded,Endorsement of someone promoting racism or oth...,noncoded
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,formal,election integrity,election integrity,racist,S. Senate spoke of ``the pernicious and nasty...,2020-01-06,Mr. BARRASSO,Senate,CREC-2021-03-03-pt1-PgS1007,None,noncoded,Voter suppression that targets people of color,noncoded
120,formal,election integrity,election integrity,racist,"Earlier this year, we witnessed only the lates...",2020-01-06,Mr. SCHUMER,Senate,CREC-2021-03-17-pt1-PgS1578-2,None,noncoded,Voter suppression that targets people of color,coded
121,formal,election integrity,election integrity,racist,One is the breakdown of trust in the system it...,2020-01-06,Mr. KING,Senate,CREC-2021-10-19-pt1-PgS7082-5,None,noncoded,Voter suppression that targets people of color,coded
122,formal,election integrity,election integrity,racist,It was so disturbing to read in the Washington...,2020-01-06,Mr. SCHUMER,Senate,CREC-2023-02-01-pt1-PgS180-2,None,noncoded,Voter suppression that targets people of color,coded


In [78]:
accuracy_disambiguation=accuracy_score(df_disambiguation['label'], df_disambiguation['predicted_label'])
f1_disambiguation=f1_score(df_disambiguation['label'], df_disambiguation['predicted_label'], pos_label='coded')
recall_disambiguation=recall_score(df_disambiguation['label'], df_disambiguation['predicted_label'], pos_label='coded')
precision_score_disambiguation=precision_score(df_disambiguation['label'], df_disambiguation['predicted_label'], pos_label='coded')
print("disambiguation_accuracy: ",accuracy_disambiguation)
print("f1_disambiguation: ", f1_disambiguation)
print("recall_disambiguation: ",recall_disambiguation)
print("precision_score_disambiguation: ",precision_score_disambiguation)

disambiguation_accuracy:  0.7580645161290323
f1_disambiguation:  0.761904761904762
recall_disambiguation:  0.7164179104477612
precision_score_disambiguation:  0.8135593220338984


In [79]:
df_disambiguation.to_csv("parsed_dog_whistle_disambiguation.csv", index=False)

### disambiguation few_shot

In [85]:
raw_disambiguation_fewshot_result=[]

In [83]:
def llm_disambiguation_few(lst_sentences, sentence_indexes, dog_whistle, definition_of_dog_whistle, lst_sentences_examples, example_sentence_indexes, example_dog_whistles, example_definition_of_dog_whistle, example_answer_indexes):
    input_definition = f"""You are a helpful assistant participating in a content moderation task of dog whistle detection.
        A dogwhistle is the use of coded or suggestive language in political messaging to garner support 
        from a particular group without provoking opposition. 
        Below are examples of the task. The dog whistle and definition will be given, followed by a list of sentences. The task is to decide which of the sentences contain the use of the dog whistle in the coded meaning. The answer is a list of the sentence indexes."""
    for i in range(len(lst_sentences_examples)):
        input_definition += f"""The coded meaning of {example_dog_whistles[i]} is: {example_definition_of_dog_whistle[i]}.
            List of sentences: {[str(example_sentence_indexes[i][key])+': ' + str(value) for key, value in enumerate(lst_sentences_examples[i])]}
            Answer: {example_answer_indexes[i]}
        """
    input_definition += f"""
    The coded meaning of {dog_whistle} is: {definition_of_dog_whistle}.  
    Can you identify which sentences in the set below are using {dog_whistle} as a dog whistle? 
    {[str(sentence_indexes[key])+': ' + str(value) for key, value in enumerate(lst_sentences)]}
    Please structure your response as a JSON object,
    where {{
    "sentences_with_dog_whistles": "<[list of indexes]>",
    "explanations": "<explanation>"
    }}.
    """
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo",
        messages=[{"role": "user", "content": input_definition}]
    )
    raw_output = response.choices[0].message.content
    raw_disambiguation_fewshot_result.append({
        "raw_disambiguation_fewshot_result": raw_output
    })

    match = re.search(r"```json\s*(.*?)```", raw_output, flags=re.DOTALL)
    cleaned = match.group(1) if match else ""
    return cleaned, raw_disambiguation_fewshot_result

In [87]:
results_disambiguation_few=[]

for dog_whistle in tqdm(np.unique(df_disambiguation['dog_whistle'])):
    definition_of_dog_whistle = list(df_disambiguation.loc[df_disambiguation['dog_whistle']==dog_whistle, 'definition'])[0]
    lst_sentences = list(df_disambiguation.loc[df_disambiguation['dog_whistle']==dog_whistle, 'content'])
    sentence_indexes = list(df_disambiguation[df_disambiguation['dog_whistle'] == dog_whistle].index)
    df_example = df_disambiguation[df_disambiguation['dog_whistle']!=dog_whistle]
    example_dog_whistles = np.random.choice(df_example['dog_whistle'].unique(), size=3, replace=False)
    lst_sentences_examples = [list(df_example.loc[df_example['dog_whistle']==x, 'content']) for x in example_dog_whistles]
    example_sentence_indexes = [list(df_example[df_example['dog_whistle'] == x].index) for x in example_dog_whistles]
    example_definition_of_dog_whistle = [list(df_example.loc[df_example['dog_whistle']==x, 'definition'])[0] for x in example_dog_whistles]
    example_answer_indexes = [list(df_example.loc[(df_example['dog_whistle'] == x) & (df_example['label'] == 'coded')].index) for x in example_dog_whistles]
    
    json_result, raw_disambiguation_fewshot_result = llm_disambiguation_few(lst_sentences, sentence_indexes, dog_whistle, definition_of_dog_whistle, lst_sentences_examples, example_sentence_indexes, example_dog_whistles, example_definition_of_dog_whistle, example_answer_indexes)
    try:
      response_dict = json.loads(json_result)
      results_disambiguation_few.append(response_dict)
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON at row {index}: {e}") 
      results_disambiguation_few.append({
      "sentences_with_dog_whistles": np.nan,
      "explanations": np.nan
    })
with open('raw_disambiguation_fewshot_result.json', 'w') as outfile:
  json.dump(raw_disambiguation_fewshot_result, outfile, indent=4)


with open('llama3.0_disambiguation_fewshot_result.json', 'w') as outfile:
  json.dump(results_disambiguation_few, outfile, indent=4)


100%|██████████| 21/21 [00:57<00:00,  2.75s/it]


In [88]:
with open('llama3.0_disambiguation_fewshot_result.json', 'r') as f:
  data = json.load(f)
disambiguation_fewshot_results = pd.DataFrame(data)
disambiguation_fewshot_results

,sentences_with_dog_whistles,explanations
0,"[105, 106]",Sentences 105 and 106 are using 'Baltimore' as...
1,[82],The sentence at index 82 uses the term 'Islami...
2,[83],The sentence is using the term 'radical Islami...
3,"[37, 38, 39, 40, 41]","The sentences with indexes 37, 38, 39, 40, and..."
4,[],The word 'echo' in sentence 6 is used in its l...
5,"[115, 118, 119, 120, 121, 122, 123]",These sentences use the term 'election integri...
6,[77],"In sentence 77, the term 'extremism' is used i..."
7,[79],The sentence is using 'extremist' as a dog whi...
8,"[80, 81]",Both sentences use the term 'extremists' in a ...
9,"[62, 63, 65]","The sentences at indexes 62, 63, and 65 contai..."


In [109]:
#disambiguation_fewshot_results.to_csv("disambiguation_fewshot_results.csv", index=False)
disambiguation_fewshot_results=pd.read_csv("disambiguation_fewshot_results.csv")

In [116]:
df_disambiguation['predicted_label_few'] = np.repeat('noncoded', repeats=len(df_disambiguation))
for row_num, row in disambiguation_fewshot_results.iterrows():
    lst_positive_preds = row['sentences_with_dog_whistles']
    if isinstance(lst_positive_preds, str):
        lst_positive_preds = ast.literal_eval(lst_positive_preds)
    lst_positive_preds = [int(index) for index in lst_positive_preds]
    df_disambiguation.loc[df_disambiguation.index.isin(lst_positive_preds), 'predicted_label_few'] = 'coded'
df_disambiguation['label'] = df_disambiguation['label'].replace({
    'non-coded': 'noncoded',
    'noncoded': 'noncoded'  # Just in case of inconsistent hyphenation
})
df_disambiguation

,type,dog_whistle,dog_whistle_root,ingroup,content,date,speaker,chamber,reference,subreddit,label,definition,predicted_label_few
0,colloquial,terrorist,Islamic terrorists,Islamophobic,Radical feminism is a terrorist organization,None,None,None,None,walkaway,noncoded,Muslim people as a whole are a threat,noncoded
1,colloquial,terrorist,Islamic terrorists,Islamophobic,There's a group of right-wing terrorists that ...,None,None,None,None,walkaway,noncoded,Muslim people as a whole are a threat,noncoded
2,colloquial,terrorist,Islamic terrorists,Islamophobic,Would be difficult to do. About the only mecha...,None,None,None,None,consoom,noncoded,Muslim people as a whole are a threat,noncoded
3,colloquial,terrorist,Islamic terrorists,Islamophobic,"Rapes, terrorist attacks, violence in general ...",None,None,None,None,walkaway,noncoded,Muslim people as a whole are a threat,noncoded
4,colloquial,based,based,white supremacist,Where are the prosecutions for a doctor taking...,None,None,None,None,walkaway,noncoded,Endorsement of someone promoting racism or oth...,noncoded
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,formal,election integrity,election integrity,racist,S. Senate spoke of ``the pernicious and nasty...,2020-01-06,Mr. BARRASSO,Senate,CREC-2021-03-03-pt1-PgS1007,None,noncoded,Voter suppression that targets people of color,coded
120,formal,election integrity,election integrity,racist,"Earlier this year, we witnessed only the lates...",2020-01-06,Mr. SCHUMER,Senate,CREC-2021-03-17-pt1-PgS1578-2,None,noncoded,Voter suppression that targets people of color,coded
121,formal,election integrity,election integrity,racist,One is the breakdown of trust in the system it...,2020-01-06,Mr. KING,Senate,CREC-2021-10-19-pt1-PgS7082-5,None,noncoded,Voter suppression that targets people of color,coded
122,formal,election integrity,election integrity,racist,It was so disturbing to read in the Washington...,2020-01-06,Mr. SCHUMER,Senate,CREC-2023-02-01-pt1-PgS180-2,None,noncoded,Voter suppression that targets people of color,coded


In [117]:
df_disambiguation.to_csv("parsed_few_disambiguation.csv",index=False)

In [93]:
accuracy_disambiguation_fewshot=accuracy_score(df_disambiguation['label'], df_disambiguation['predicted_label'])
f1_disambiguation_fewshot=f1_score(df_disambiguation['label'], df_disambiguation['predicted_label'], pos_label='coded')
recall_disambiguation_fewshot=recall_score(df_disambiguation['label'], df_disambiguation['predicted_label'], pos_label='coded')
precision_score_disambiguation_fewshot=precision_score(df_disambiguation['label'], df_disambiguation['predicted_label'], pos_label='coded')
print("disambiguation_fewshot_accuracy: ",accuracy_disambiguation_fewshot)
print("f1_disambiguation_fewshot: ", f1_disambiguation_fewshot)
print("recall_disambiguation_fewshot: ",recall_disambiguation_fewshot)
print("precision_score_disambiguation_fewshot: ",precision_score_disambiguation_fewshot)

disambiguation_fewshot_accuracy:  0.7983870967741935
f1_disambiguation_fewshot:  0.8031496062992126
recall_disambiguation_fewshot:  0.7611940298507462
precision_score_disambiguation_fewshot:  0.85


In [94]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine